In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase

In [2]:
kb = KnowledgeBase(yaml_file="test.yaml")

In [3]:
kb._predicates

{'Iris': Ellipsis,
 'Iris_2': Ellipsis,
 'Only_one_of_3': Only_one_of_3(),
 'Only_one_of_2': Only_one_of_2()}

In [4]:
kb._loaders

[ <Loader>:(['flower'] -> ['type']),  <Loader>:(['flower2'] -> ['type_2'])]

In [5]:
kb.rule_to_data_loader_mapping

{'forall flower. exists c. (flower[class] = c) <-> Iris(flower, c)': [ <Loader>:(['flower'] -> ['type'])],
 'forall flower2. Iris(flower, c) and Only_one_of_3(c, setosa, versicolor, virginica)': [ <Loader>:(['flower2'] -> ['type_2']),
   <Loader>:(['flower'] -> ['type'])]}

In [6]:
kb.rules

[∀ flower.(∃ c.(((flower[class] = c) <-> Iris(flower, c)))),
 ∀ flower2.((Iris(flower, c) & Only_one_of_3(c, setosa, versicolor, virginica)))]

In [7]:
kb.rules[0]

∀ flower.(∃ c.(((flower[class] = c) <-> Iris(flower, c))))